In [ ]:
import pandas as pd

In [ ]:
df_rel = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/LDA/df_rel_cleantech.json')
df_uspto = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/LDA/df_uspto_cleantech.json')
df_epo = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/LDA/df_epo_cleantech.json')

In [ ]:
# Concatenate the three dataframes
df_cleantech = pd.concat([df_rel, df_uspto, df_epo], ignore_index=True)

In [ ]:
df_cleantech.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/LDA/df_epo_uspto_rel_cleantech.json')